🌱 Perfil Ideal de Solo e Clima para as Plantações

🌱 Análise do Perfil Ideal de Solo/Clima por Cultura (Sem Outliers)

Este script realiza a limpeza de dados e a visualização comparativa do perfil ideal de solo e clima de diferentes culturas agrícolas com base em variáveis do solo e do clima, utilizando um gráfico de radar para facilitar a interpretação.

In [ ]:
#🌱 Análise do Perfil Ideal de Solo/Clima por Cultura (Sem Outliers)

# Remoção de outliers usando o método do IQR
def remover_outliers_iqr(df, colunas):
    df_limpo = df.copy()
    for col in colunas:
        Q1 = df_limpo[col].quantile(0.25)
        Q3 = df_limpo[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df_limpo = df_limpo[(df_limpo[col] >= limite_inferior) & (df_limpo[col] <= limite_superior)]
    return df_limpo

# Variáveis numéricas
variaveis_numericas = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

# Aplicar remoção de outliers
df_sem_outliers = remover_outliers_iqr(df, variaveis_numericas)

# Calcular perfil ideal sem outliers
perfil_ideal_sem_outliers = df_sem_outliers.groupby("label")[variaveis_numericas].mean().round(2)

# Exibir
print("📋 Perfil ideal de solo/clima por cultura (sem outliers):")
print(perfil_ideal_sem_outliers)

# Gráfico de radar para o perfil ideal de solo/clima por cultura (sem outliers)

def plot_radar_todas_culturas_sem_marcadores(df):
    categorias = list(df.columns)
    num_vars = len(categorias)
    angulos = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angulos += angulos[:1]

    plt.figure(figsize=(12, 9))
    ax = plt.subplot(111, polar=True)
    plt.xticks(angulos[:-1], categorias, size=12)

    cores = [
        '#FF7732', '#FE9232', '#CCC33F', '#ADCC3B', '#91902C',
        '#59AE31', '#00FF58', '#007656', '#35D495', '#00ADA4',
        '#3700FD', '#00A9C2', '#39E2D6', '#69A0F4', '#9083F4',
        '#9300FF', '#B200FF', '#CF38AC', '#CF6A6A', '#FF0000'
    ]

    estilos_linha = ['solid', 'dashed', 'dotted', 'dashdot']

    for i, cultura in enumerate(df.index):
        valores = df.loc[cultura].values.flatten().tolist()
        valores += valores[:1]

        cor = cores[i % len(cores)]
        estilo = estilos_linha[i % len(estilos_linha)]

        ax.plot(
            angulos, valores, linewidth=2.5, linestyle=estilo, label=cultura,
            color=cor, alpha=0.95
        )
        # Sem scatter (sem marcadores)

    plt.title('🌾 Perfil ideal de solo/clima por cultura', size=14, fontname='Segoe UI Emoji')
    plt.legend(loc='upper right', bbox_to_anchor=(1.35, 1.1), fontsize=12)
    plt.tight_layout()
    plt.show()

plot_radar_todas_culturas_sem_marcadores(perfil_ideal_sem_outliers)

🧪 Análise do Perfil Ideal de Culturas Agrícolas (com Radar Plot)

Este script realiza uma análise exploratória de dados agrícolas com o objetivo de identificar o perfil ideal de solo e clima para culturas comuns, baseado em registros reais e sem distorções causadas por outliers ou culturas com poucos dados.

✅ Resultado
O gráfico radar representa visualmente o perfil médio ideal para cultivo de culturas agrícolas, sem a influência de outliers ou amostras raras, pode ser usado como base para comparar culturas específicas e identificar onde estão alinhadas ou distantes do ideal.

In [ ]:
#🧪 Análise do Perfil Ideal de Culturas Agrícolas (com Radar Plot)

# Remoção de outliers usando o método do IQR
def remover_outliers_iqr(df, colunas):
    df_limpo = df.copy()
    for col in colunas:
        Q1 = df_limpo[col].quantile(0.25)
        Q3 = df_limpo[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df_limpo = df_limpo[(df_limpo[col] >= limite_inferior) & (df_limpo[col] <= limite_superior)]
    return df_limpo

# Variáveis numéricas
variaveis_numericas = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']

# Aplicar remoção de outliers
df_sem_outliers = remover_outliers_iqr(df, variaveis_numericas)

# Calcular média geral das variáveis numéricas, agrupado por cultura
perfil_medio = df_sem_outliers.groupby("label")[variaveis_numericas].mean().round(2)

# Remover culturas consideradas outliers — vamos definir "culturas outlier" como aquelas com poucos registros (por exemplo, menos que 30 amostras)
contagem_culturas = df_sem_outliers['label'].value_counts()
culturas_validas = contagem_culturas[contagem_culturas >= 30].index.tolist()
perfil_filtrado = perfil_medio.loc[culturas_validas]

# Calcular o perfil médio geral para essas culturas filtradas
perfil_ideal = perfil_filtrado.mean().round(2)

# Preparar dados para gráfico radar
variaveis = perfil_ideal.index.tolist()
valores = perfil_ideal.values.tolist()

# Fechar ciclo do radar (repete o primeiro ponto)
variaveis += [variaveis[0]]
valores += [valores[0]]


# Criar gráfico radar com Plotly e Exibir

# Exibir perfil ideal
print("🌱 Perfil ideal de solo/clima (média das culturas não-outliers):")
display(perfil_ideal.to_frame(name='valor'))


# Gráfico radar interativo
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=valores,
    theta=variaveis,
    fill='toself',
    name='Perfil Ideal Médio',
    marker=dict(color='mediumseagreen'),
    hoverinfo='text',
    text=[f'{v:.2f}' for v in valores]
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, max(valores)*1.2]  # ajustar o range para melhor visualização
        )
    ),
    title="Perfil Ideal de Solo/Clima para Culturas Agrícolas (sem Outliers e Culturas Raras)",
    showlegend=True,
    height=600,
    width=600
)

fig.show()

🌱 Culturas Não-Outliers

Este trecho de código exibe as culturas agrícolas consideradas não-outliers após a remoção dos valores extremos do conjunto de dados.

In [ ]:
#🌱 Culturas Não-Outliers

# Culturas não-outliers
print("🌱 Culturas não-outliers:")
print(df_sem_outliers[df_sem_outliers['label'].isin(culturas_validas)]['label'].value_counts())

🧭 Gráfico radar interativo (Plotly) para arroz, milho, café e perfil de solo ideal

Este gráfico do tipo radar compara as médias das variáveis numéricas para os cultivos de arroz, milho e café, além de um perfil de solo ideal definido previamente. Cada cultura apresenta um perfil específico, com destaque para diferentes variáveis como precipitação, umidade, nitrogênio, fósforo, potássio, pH e temperatura. O perfil ideal serve como referência para identificar o quão próximo cada cultura está das condições consideradas ideais. Essa visualização facilita a análise multivariada e pode apoiar decisões agronômicas, como sugestões de cultivo para determinadas regiões, ajustes no solo ou estratégias de manejo baseadas em similaridade com o perfil ideal.

In [ ]:
#🧭 Gráfico radar interativo (Plotly) para arroz, milho, café e perfil ideal

# Médias por cultura
df_rice = df[df['label'] == 'rice'].drop(columns='label').mean()
df_maize = df[df['label'] == 'maize'].drop(columns='label').mean()
df_coffee = df[df['label'] == 'coffee'].drop(columns='label').mean()

# Valores em lista
val_rice = df_rice.values.tolist()
val_maize = df_maize.values.tolist()
val_coffee = df_coffee.values.tolist()
val_ideal = perfil_ideal.values.tolist()

# Fechar ciclo do radar (primeiro ponto se repete no final)
variaveis = perfil_ideal.index.tolist()
variaveis += [variaveis[0]]
val_rice += [val_rice[0]]
val_maize += [val_maize[0]]
val_coffee += [val_coffee[0]]
val_ideal += [val_ideal[0]]

# Criar figura Plotly
fig = go.Figure()

# Arroz
fig.add_trace(go.Scatterpolar(
    r=val_rice,
    theta=variaveis,
    fill='toself',
    name='Arroz',
    marker=dict(color='green'),
    hoverinfo='text',
    text=[f'{v:.1f}' for v in val_rice]
))

# Milho
fig.add_trace(go.Scatterpolar(
    r=val_maize,
    theta=variaveis,
    fill='toself',
    name='Milho',
    marker=dict(color='orange'),
    hoverinfo='text',
    text=[f'{v:.1f}' for v in val_maize]
))

# Café
fig.add_trace(go.Scatterpolar(
    r=val_coffee,
    theta=variaveis,
    fill='toself',
    name='Café',
    marker=dict(color='brown'),
    hoverinfo='text',
    text=[f'{v:.1f}' for v in val_coffee]
))

# Perfil Ideal
fig.add_trace(go.Scatterpolar(
    r=val_ideal,
    theta=variaveis,
    fill='toself',
    name='Perfil Ideal',
    marker=dict(color='cyan'),
    hoverinfo='text',
    text=[f'{v:.1f}' for v in val_ideal]
))

# Layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    title="Perfil Radar Interativo - Arroz, Milho, Café e Perfil Ideal",
    showlegend=True,
    height=700,
    width=700
)

# Exibir
fig.show()

📊 Relatório – Comparativo: Arroz, Milho e Café vs Perfil Ideal

Este relatório apresenta, para cada cultura (arroz, milho e café), as condições agrícolas ideais de solo e clima, comparando-as ao Perfil Ideal genérico — que serve apenas como referência das exigências médias da maioria das culturas e não deve ser seguido rigorosamente em todos os casos. A análise detalha temperatura, pH, umidade, precipitação e manejo de nutrientes (N, P, K), evidenciando quais ajustes são necessários para otimizar cada cultivo.

🌾 Arroz

Temperatura: O arroz alcança desempenho máximo em uma temperatura média de 25 °C, próximo dos valores ideais.
pH do Solo: Prefere solos ligeiramente ácidos a neutros, com pH médio de 6,5, garantindo a disponibilidade de nutrientes, assim como a maioria das culturas.
Precipitação e Umidade: Necessita de rios caudalosos ou áreas alagadas e precipitação anual elevada, mantendo o solo encharcado; a umidade próxima de 100% favorece seu desenvolvimento.
Nitrogênio (N): Apresenta níveis altos em comparação com o perfil ideal, indicando que o arroz precisa de solos ricos em nitrogênio para se desenvolver.
Fósforo (P) e Potássio (K): O arroz geralmente necessita de quantidades moderadas a altas de P e K, conforme a caracterização do solo local; solos pobres em fósforo limitam o rendimento.

🌽 Milho

Temperatura: Mais eficiente em áreas com temperatura média de 25 °C, típica de zonas temperadas e subtropicais.
pH do Solo: Ideal com valor de 6,5, equilibrando a disponibilidade de nutrientes e a atividade microbiana, de acordo com os padrões ideais.
Precipitação e Umidade: Prefere solos leves, bem drenados e ricos em matéria orgânica.
Nitrogênio (N): Apresenta níveis altos em comparação com o perfil ideal, indicando que o milho precisa de solos ricos em nitrogênio para se desenvolver.
Fósforo (P) e Potássio (K): Necessita de quantidades moderadas a altas de P e K.

☕ Café

Temperatura: Alinhada com o perfil ideal, na faixa de 25 °C.
pH do Solo: Prefere solos ligeiramente ácidos, com pH de 6,5, favorecendo a absorção de micronutrientes.
Precipitação e Umidade: Necessita de precipitação acima dos níveis ideais, mas não tanto quanto o arroz. A umidade está alinhada com o esperado para a maioria das culturas.
Nitrogênio (N): Apresenta os níveis mais altos em comparação com o perfil ideal, necessitando de solos ricos em nitrogênio para se desenvolver.
Fósforo (P) e Potássio (K): Os níveis de fósforo são menores que os do solo ideal, enquanto o potássio está alinhado com o ideal.

✅ Conclusões Finais

O Perfil Ideal é uma referência média e não substitui a adaptação das práticas agrícolas.

- Arroz: Otimizado em solos alagados e ácidos, com manejo cuidadoso de nitrogênio e drenagem controlada.
- Milho: Requer solo bem drenado e manejo balanceado de fósforo e potássio.
- Café: Beneficia-se de altos níveis de precipitação e nitrogênio.